<a href="https://colab.research.google.com/github/joshuaghannan/ECEC247_Project/blob/master/Updated_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
import pywt

### Set up the Device

In [2]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    # device = torch.device("cuda:1") # For Yiming 
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


### If Using Colab

In [3]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

Eden_tests1.ipynb  person_test.npy	   X_test.npy
Eden_tests2.ipynb  person_train_valid.npy  X_train_valid.npy
EEG_loading.ipynb  __pycache__		   y_test.npy
FinalProject	   X_test_filtered.npy	   y_train_valid.npy


### Load the Datasets

In [5]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


# Data Manipulation

### K-Fold

In [0]:
# some major changes here for the Train_Val_Data function
def Train_Val_Data(X_train_valid, y_train_val):
    '''
    split the train_valid into k folds (we fix k = 5 here)
    return: list of index of train data and val data of k folds
    train_fold[i], val_fold[i] is the index for training and validation in the i-th fold 

    '''
    fold_idx = []
    train_fold = []
    val_fold = []
    train_val_num = X_train_valid.shape[0]
    fold_num = int(train_val_num / 5)
    perm = np.random.permutation(train_val_num)
    for k in range(5):
        fold_idx.append(np.arange(k*fold_num, (k+1)*fold_num, 1))
    for k in range(5):
        val_fold.append(fold_idx[k])
        count = 0
        for i in range(5):
            if i != k:
                if count == 0:
                    train_idx = fold_idx[i]
                else:
                    train_idx = np.concatenate((train_idx, fold_idx[i]))
                count += 1
        train_fold.append(train_idx)

    return train_fold, val_fold

### Customized Dataset

In [0]:
class EEG_Dataset(Dataset):
    '''
    use use fold_idx to instantiate different train val datasets for k-fold cross validation

    '''
    def __init__ (self, X_train=None, y_train=None, p_train=None, X_val=None, y_val=None, p_val=None, X_test=None, y_test=None, p_test=None, mode='train'):
        if mode == 'train':
            self.X = X_train
            self.y = y_train- 769
            self.p = p_train
            
        elif mode == 'val':
            self.X = X_val
            self.y = y_val- 769
            self.p = p_val

        elif mode == 'test':
            self.X = X_test
            self.y = y_test - 769        
            self.p = p_test

    def __len__(self):
        return (self.X.shape[0])
    
    def __getitem__(self, idx):
        '''
        X: (augmented) time sequence 
        y: class label
        p: person id

        '''
        X = torch.from_numpy(self.X[idx,:,:]).float()
        y = torch.tensor(self.y[idx]).long()
        p = torch.tensor(self.p[idx]).long()
        #p = torch.from_numpy(self.p[idx,:]).long()     
        sample = {'X': X, 'y': y, 'p':p}

        return sample

## Data Augmentation Functions

###Center and Whiten Data
Scales and shifts data to have zero mean and variance 1

In [0]:
from sklearn import preprocessing
def scale_data(X):
  #Takes 3-dim X and outputs scaled and shifted X_new with zero mean and var 1
  X_scaled = np.empty_like(X)
  for i in range(X.shape[1]):
    X_scaled[:,i,:] = preprocessing.scale(X[:,i,:])
  return X_scaled

### 1. Window Data

In [0]:
def window_data(X, y, p, window_size, stride):
  '''
  X (a 3-d tensor) of size (#trials, #electrodes, #time series)
  y (#trials,): label 
  p (#trials, 1): person id

  X_new1: The first output stacks the windowed data in a new dimension, resulting 
    in a 4-d tensor of size (#trials x #electrodes x #windows x #window_size).
  X_new2: The second option makes the windows into new trails, resulting in a new
    X tensor of size (#trials*#windows x #electrodes x #window_size). To account 
    for the larger number of trials, we also need to augment the y data.
  y_new: The augmented y vector of size (#trials*#windows) to match X_new2.
  p_new: The augmented p vector of size (#trials*#windows) to match X_new2
 
  '''
  num_sub_trials = int((X.shape[2]-window_size)/stride)
  X_new1 = np.empty([X.shape[0],X.shape[1],num_sub_trials,window_size])
  X_new2 = np.empty([X.shape[0]*num_sub_trials,X.shape[1],window_size])
  y_new = np.empty([X.shape[0]*num_sub_trials])
  p_new = np.empty([X.shape[0]*num_sub_trials])
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      for k in range(num_sub_trials):
        X_new1[i,j,k:k+window_size]    = X[i,j,k*stride:k*stride+window_size]
        X_new2[i*num_sub_trials+k,j,:] = X[i,j,k*stride:k*stride+window_size]
        y_new[i*num_sub_trials+k] = y[i]
        p_new[i*num_sub_trials+k] = p[i]
  return X_new1, X_new2, y_new, p_new

### 2. STFT

In [0]:
# Function that computes the short-time fourier transform of the data and returns the spectrogram
def stft_data(X, window, stride):
    '''
    Inputs:
    X - input data, last dimension is one which transform will be taken across.
    window - size of sliding window to take transform across
    stride - stride of sliding window across time-series

    Returns:
    X_STFT - Output data, same shape as input with last dimension replaced with two new dimensions, F x T.
            where F = window//2 + 1 is the frequency axis
            and T = (input_length - window)//stride + 1, similar to the formula for aconvolutional filter.
    t - the corresponding times for the time axis, T
    f - the corresponding frequencies on the frequency axis, F.

    reshape X_STFT (N, C, F, T) to (N, C*F, T) to fit the input of rnn

    Note that a smaller window means only higher frequencies may be found, but give finer time resolution.
    Conversely, a large window gives better frequency resolution, but poor time resolution.

    '''
    noverlap = window-stride
    #print(noverlap)
    if noverlap < 0 :
        print('Stride results in skipped data!')
        return
    f, t, X_STFT = sig.spectrogram(X,nperseg=window,noverlap=noverlap,fs=250, return_onesided=True)
    N, C, F, T = X_STFT.shape
    X_STFT = X_STFT.reshape(N, C*F, T)
    return X_STFT

### 3. CWT

In [0]:
def cwt_data(X, num_levels, top_scale=1.5):
    '''
    Takes in data, computes CWT using the mexican hat or ricker wavelet using scipy
    Also takes in the top scale parameter.  I use logspace, so scale goes from 1 -> 2^top_scale with num_levels steps.
    Appends to the data a new dimension, of size 'num_levels'
    New dimension corresponds to wavelet content at num_levels different scalings (linear)
    also returns the central frequencies that the scalings correspond to
    input data is N x C X T
    output data is N x C x T x F
    note: CWT is fairly slow to compute

    # EXAMPLE USAGE
    test, freqs = cwt_data(X_train_valid[0:5,:,:],num_levels=75,top_scale=4)
    '''
    scales = np.logspace(start=0,stop=top_scale,num=num_levels)
    out = np.empty((X.shape[0],X.shape[1],X.shape[2],num_levels))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            coef = sig.cwt(X[i,j,:],sig.ricker,scales)
            out[i,j,:] = coef.T
    freqs = pywt.scale2frequency('mexh',scales)*250
    N, C, T, F = out.shape
    X_CWT = np.transpose(out, (0,1,3,2)).reshape(N, C*F, T)
    return X_CWT

In [0]:
def cwt_data2(X, y, p, num_levels, bottom_scale = 0.1, top_scale=0.4):
    '''
    Takes in data, computes CWT using the mexican hat or ricker wavelet using scipy
    Also takes in the top scale parameter.  I use logspace, so scale goes from 1 -> 2^top_scale with num_levels steps.
    Appends to the data a new dimension, of size 'num_levels'
    New dimension corresponds to wavelet content at num_levels different scalings (linear)
    also returns the central frequencies that the scalings correspond to
    input data is N x C X T
    output data is N x C x T x F
    note: CWT is fairly slow to compute

    # EXAMPLE USAGE
    test, freqs = cwt_data(X_train_valid[0:5,:,:],num_levels=75,top_scale=4)
    '''
    scales = np.logspace(start=bottom_scale, stop=top_scale,num=num_levels)
    out = np.empty((X.shape[0],X.shape[1],X.shape[2],num_levels))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            coef = sig.cwt(X[i,j,:],sig.ricker,scales)
            out[i,j,:] = coef.T
    freqs = pywt.scale2frequency('mexh',scales)*250
    N, C, T, F = out.shape
    X_cwt = np.transpose(out, (0,3,1,2)).reshape(N*F, C, T)
    y_cwt = np.empty([X.shape[0]*F])
    p_cwt = np.empty([X.shape[0]*F])
    for i in range(X.shape[0]):
      for k in range(F):
        y_cwt[i*F+k] = y[i]
        p_cwt[i*F+k] = p[i]
    return X_cwt, y_cwt, p_cwt

### 4. Independent Component Analysis (ICA)

In [0]:
def ica_data(X, n_components):
  # FUNCTION TO COMPUTE THE ICA OF DATA
  out = np.empty((X.shape[0], n_components, X.shape[-1]))
  ica = FastICA(n_components=n_components, whiten=True, max_iter=300, tol=0.01)
  for i in range(X.shape[0]):
    out[i,:,:] = ica.fit_transform(X[i,:,:].T).T  # Reconstruct signals
  return out

## Define data augmentation wrapper

In [0]:
def Aug_Data(X, y, p, aug_type=None, window_size=200, window_stride=20, stft_size=None, stft_stride=None, cwt_level=None, cwt_scale=None):
    if aug_type == None:
        X_aug, y_aug, p_aug = X, y, p
    elif aug_type == "window":
        _, X_aug, y_aug, p_aug = window_data(X, y, p, window_size, window_stride)
    elif aug_type == "stft":
        X_aug = stft_data(X, stft_size, stft_stride)
        y_aug, p_aug = y, p
    elif aug_type == 'cwt':
        X_aug = cwt_data(X, cwt_level, cwt_scale)
        y_aug, p_aug = y, p
    elif aug_type == 'cwt2':
        X_aug, y_aug, p_aug = cwt_data2(X, y, p, cwt_level, cwt_scale)
    
    return X_aug, y_aug, p_aug

In [15]:
window_size=250
vote_num = 50
vote_idx = np.random.choice(1000-window_size, vote_num)
print(max(vote_idx))

726


# Architectures

### Define Basic LSTM

In [0]:
class LSTMnet(nn.Module):
    '''
    Create Basic LSTM:
    2 layers

    TODO: make number of layers, dropout, activation function, regularization all params
    see ex: https://blog.floydhub.com/gru-with-pytorch/
    '''

    def __init__(self, input_size, hidden_size, output_dim, dropout):
        super(LSTMnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.fc(out[-1, :, :])
        return out

### Define Basic GRU

In [0]:
class GRUnet(nn.Module):
    '''
    Create Basic GRU:
    2 layers

    TODO: make number of layers, dropout, activation function, regularization all params
    see ex: https://blog.floydhub.com/gru-with-pytorch/
    '''

    def __init__(self, input_size, hidden_size, output_dim, dropout):
        super(GRUnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.fc(out[-1, :, :])
        return out

# RNN Initialization

In [0]:
def InitRNN(rnn_type="LSTM", input_size=22, hidden_size=50, output_dim=4, dropout=0.5, lr=1e-3):
    '''
    Function to initialize RNN
    
    input: RNN type(LSTM, GRU), and other params if neccessary (regularization, acitvation, dropout, num layers, etc.)

    output: model, criterion, optimizer

    TODO: Eventually should also take in params such as dropout, number of layers, and activation function(s), etc.
    '''

    if rnn_type=="LSTM":
        model = LSTMnet(input_size=input_size, hidden_size=hidden_size, output_dim=output_dim, dropout=dropout).to(device)

    elif rnn_type=="GRU":
        model = GRUnet(input_size=input_size, hidden_size=hidden_size, output_dim=output_dim, dropout=dropout).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer


### K-Fold Training and Cross Validation

In [0]:
def TrainRNN(trainloader, valloader, num_epochs=20, verbose=True, aug_type=None):
    val_acc_list = []
    for ep in range(num_epochs):
        tstart = time.time()
        running_loss = 0.0
        correct, total = 0, 0
        for idx, batch in enumerate(EEG_trainloader):
            optimizer.zero_grad()
            X = batch['X'].permute(2, 0, 1).to(device)
            y = batch['y'].to(device)
            output = model(X)
            loss = criterion(output, y)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            pred = torch.argmax(output, dim=1)
            correct += torch.sum(pred == y).item()
            total += y.shape[0]
        train_acc = correct / total
        train_loss = running_loss
        '''
        The validation need to be customized according to the data augmenation type
        for stft and cwt: they didn't increase the number of trials, we can directly pass the augmented data to the model
        for window: it increase the number of trials, we need to do a voting for different subsequences in one trial
        
        '''
        if aug_type == 'window':
            correct, total = 0, 0
            for idx, batch in enumerate(EEG_valloader):
                X = batch['X'].permute(2, 0, 1).to(device)
                #X = batch['X'].to(device)
                y = batch['y'].to(device)
                vote_idx = np.random.choice(1000-window_size, vote_num)
                vote_pred = np.zeros(y.shape[0])
                for i in range(len(vote_idx)):
                    X_sub = X[vote_idx[i]:vote_idx[i]+window_size,:,:]
                    output = model(X_sub)
                    pred = torch.argmax(output, dim=1)
                    if i == 0:
                        vote_matrix = np.asarray(pred.cpu().view(-1, 1))
                    else:
                        vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
                for row in range(y.shape[0]):
                    vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
                vote_pred = torch.from_numpy(vote_pred).long()
                correct += torch.sum(vote_pred == y.cpu()).item()
                total += y.shape[0]
            val_acc = correct / total      
        if aug_type == 'cwt2':
            correct, total = 0, 0
            for idx, batch in enumerate(EEG_valloader):
                X = batch['X'].permute(2, 0, 1).to(device)
                #X = batch['X'].to(device)
                y = batch['y'].to(device)
                vote_idx = np.linspace(0, X.shape[0]-1, X.shape[0]/num_levels)
                y_new = y[vote_idx]
                vote_pred = np.zeros(y.shape[0]/num_levels)
                for i in range(len(vote_idx)):
                    X_sub = X[:,vote_idx[i]:vote_idx[i]+num_levels,:]
                    output = model(X_sub)
                    pred = torch.argmax(output, dim=1)
                    if i == 0:
                        vote_matrix = np.asarray(pred.cpu().view(-1, 1))
                    else:
                        vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
                    for row in range(y_new.shape[0]):
                        vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
                vote_pred = torch.from_numpy(vote_pred).long()
                correct += torch.sum(vote_pred == y_new.cpu()).item()
                total += y.shape[0]/num_levels
            val_acc = correct / total        
        else:
            correct, total = 0, 0
            for idx, batch in enumerate(EEG_valloader):
                X = batch['X'].permute(2, 0, 1).to(device)
                y = batch['y'].to(device)
                output = model(X)                    
                pred = torch.argmax(output, dim=1)
                correct += torch.sum(pred == y.cpu()).item()
                total += y.shape[0]
            val_acc = correct / total
        tend = time.time()
        if verbose:
            print('epoch: {:<3d}    time: {:<3.2f}    loss: {:<3.3f}    train acc: {:<1.3f}    val acc: {:<1.3f}'.format(ep+1, tend - tstart, train_loss, train_acc, val_acc))
        val_acc_list.append(val_acc)
    best_val_acc = max(val_acc_list)
    return best_val_acc

# Pipeline

## 1. Split the data to train and validation

In [21]:
train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)
X_train_valid[train_fold[0]].shape

(1692, 22, 1000)

## 2. Initialize the model

In [0]:
# indicate hyperparameters here
model, criterion, optimizer = InitRNN(rnn_type='LSTM')


#Experiments

## Windowed Augmentation

### finding ideal stride for windowed augmentation

In [0]:
aug_type = "window"
window_size = 150
strides = [10, 25, 50, 100]
#win_stride = 50
vote_num = 50
best_val_acc = 0.0
k = 0
for win_stride in strides:
    model, criterion, optimizer = InitRNN(rnn_type='LSTM')
    # indicate hyperparameters here
    print ('stride: {}'.format(win_stride))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=win_stride)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_val_acc += TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
print ('best validation is :{}'.format(best_val_acc))

stride: 10
epoch: 1      time: 331.18    loss: 1553.087    train acc: 0.276    val acc: 0.262
epoch: 2      time: 327.14    loss: 1552.710    train acc: 0.280    val acc: 0.350
epoch: 3      time: 326.42    loss: 1544.233    train acc: 0.297    val acc: 0.326
epoch: 4      time: 327.20    loss: 1538.230    train acc: 0.306    val acc: 0.338
epoch: 5      time: 326.28    loss: 1534.997    train acc: 0.313    val acc: 0.355
epoch: 6      time: 324.37    loss: 1494.675    train acc: 0.358    val acc: 0.366
stride: 25
epoch: 1      time: 144.98    loss: 622.750    train acc: 0.267    val acc: 0.336
epoch: 2      time: 143.28    loss: 620.345    train acc: 0.285    val acc: 0.340
epoch: 3      time: 143.05    loss: 619.238    train acc: 0.291    val acc: 0.288
epoch: 4      time: 143.65    loss: 620.260    train acc: 0.288    val acc: 0.284
epoch: 5      time: 142.09    loss: 619.223    train acc: 0.291    val acc: 0.352
epoch: 6      time: 145.43    loss: 614.616    train acc: 0.311    val

### finding ideal window size for windowed augmentation

In [0]:
aug_type = "window"
#window_size = 50
windows = [50, 150, 200, 250]
stride = 10
vote_num = 50
best_val_acc = 0.0
k = 0
for window_size in windows:
    model, criterion, optimizer = InitRNN(rnn_type='LSTM')
    # indicate hyperparameters here
    print ('window_size {}'.format(window_size))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_val_acc = TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
#print ('best validation is :{}'.format(best_val_acc))

window_size 50
epoch: 1      time: 140.25    loss: 1733.324    train acc: 0.277    val acc: 0.357
epoch: 2      time: 137.34    loss: 1717.255    train acc: 0.306    val acc: 0.333
epoch: 3      time: 138.59    loss: 1699.115    train acc: 0.326    val acc: 0.291
epoch: 4      time: 137.23    loss: 1683.486    train acc: 0.340    val acc: 0.381
epoch: 5      time: 139.93    loss: 1667.062    train acc: 0.355    val acc: 0.388
epoch: 6      time: 138.08    loss: 1650.986    train acc: 0.368    val acc: 0.355
window_size 150
epoch: 1      time: 328.67    loss: 1554.043    train acc: 0.272    val acc: 0.253
epoch: 2      time: 326.29    loss: 1545.088    train acc: 0.291    val acc: 0.355
epoch: 3      time: 320.84    loss: 1499.160    train acc: 0.349    val acc: 0.345
epoch: 4      time: 325.38    loss: 1430.609    train acc: 0.401    val acc: 0.371
epoch: 5      time: 325.86    loss: 1359.511    train acc: 0.442    val acc: 0.392
epoch: 6      time: 322.28    loss: 1286.476    train ac

In [0]:
aug_type = "window"
#window_size = 50
windows = [100, 200, 500]
stride = 10
best_val_acc = 0.0
k = 0
for window_size in windows:
    model, criterion, optimizer = InitRNN(rnn_type='LSTM')
    # indicate hyperparameters here
    print ('window_size {}'.format(window_size))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_val_acc += TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
print ('best validation is :{}'.format(best_val_acc))

window_size 100
epoch: 1      time: 67.43    loss: 329.843    train acc: 0.259    val acc: 0.251
epoch: 2      time: 67.25    loss: 328.945    train acc: 0.275    val acc: 0.331
epoch: 3      time: 68.59    loss: 328.265    train acc: 0.288    val acc: 0.265
epoch: 4      time: 68.10    loss: 328.877    train acc: 0.276    val acc: 0.284
epoch: 5      time: 67.83    loss: 326.786    train acc: 0.298    val acc: 0.296
epoch: 6      time: 68.33    loss: 326.647    train acc: 0.300    val acc: 0.310
window_size 200
epoch: 1      time: 320.78    loss: 293.701    train acc: 0.263    val acc: 0.312
epoch: 2      time: 148.88    loss: 292.625    train acc: 0.284    val acc: 0.329
epoch: 3      time: 100.00    loss: 291.648    train acc: 0.291    val acc: 0.305
epoch: 4      time: 101.20    loss: 291.135    train acc: 0.298    val acc: 0.333
epoch: 5      time: 99.46    loss: 290.517    train acc: 0.305    val acc: 0.364
epoch: 6      time: 100.77    loss: 289.986    train acc: 0.307    val ac

## Split CWT augmentation into individual trials

#### Split the data to train and validation

In [20]:
num_levels = 5
top_scale = 0.4
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

(8460, 22, 1000)

###Run the thing

In [0]:
aug_type = 'window'
best_val_acc = 0.0
window_size = 200
stride = 50
vote_num = 50
k = 0
    
# indicate hyperparameters here
model, criterion, optimizer = InitRNN(rnn_type='LSTM', input_size = 22)
print ('fold {}'.format(k+1))
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type, cwt_level=num_levels, cwt_scale=top_scale)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
best_val_acc = TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
t ('best validation is :{}'.format(best_val_acc))

fold 1


###Test it

In [0]:
X_cwt_test, y_cwt_test, p_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)


if aug_type == 'window':
    EEG_testset = EEG_Dataset(X_train, y_train, p_train, X_val, y_val, p_val, X_cwt_test, y_cwt_test, p_cwt_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].permute(2, 0, 1).to(device)
        y = batch['y'].to(device)
        vote_idx = np.random.choice(1000-window_size, vote_num)
        vote_pred = np.zeros(y.shape[0])
        for i in range(len(vote_idx)):
            X_sub = X[vote_idx[i]:vote_idx[i]+window_size,:,:]
            output = model(X_sub)
            pred = torch.argmax(output, dim=1)
            if i == 0:
                vote_matrix = np.asarray(pred.cpu().view(-1, 1))
            else:
                vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
            for row in range(y.shape[0]):
                vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
        vote_pred = torch.from_numpy(vote_pred).long()
        correct += torch.sum(vote_pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total 
else:
    X_test, y_test, p_test = Aug_Data(X_test, y_test, p_test, aug_type=aug_type, cwt_level=num_levels, cwt_scale=top_scale)
    EEG_testset = EEG_Dataset(X_test=X_test, y_test=y_test, p_test=p_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)    
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].permute(2, 0, 1).to(device)
        y = batch['y'].to(device)
        output = model(X)                    
        pred = torch.argmax(output, dim=1)
        correct += torch.sum(pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total
print ('Testing Accuracy: {:.4f}'.format(test_acc))

Testing Accuracy: 0.4871


In [0]:
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, cwt_level=num_levels, cwt_scale=top_scale)

In [0]:
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type, cwt_level=num_levels, cwt_scale=top_scale)

In [0]:
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

In [0]:
print('running cwt2 with num_levels: {}  and top_scale: {}'.format(num_levels, top_scale))
best_val_acc = TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
print ('best validation accuracy is :{}'.format(best_val_acc))

running cwt2 with num_levels: 10  and top_scale: 0.9


KeyboardInterrupt: ignored

## CWT augmentation followed by windowing

####Preprocess data

In [0]:
num_levels = 4
top_scale = 0.5
X_cwt = cwt_data(X_train_valid, num_levels, top_scale=top_scale)

In [24]:
train_fold, val_fold = Train_Val_Data(X_cwt, y_train_valid)
X_cwt[train_fold[0]].shape

(1692, 88, 1000)

###Run the thing

In [0]:
aug_type = 'window'
best_val_acc = 0.0
window_size = 200
stride = 100
vote_num = 50
k = 0
    
for k in range(1):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='LSTM', input_size = 22*num_levels)
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_cwt[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_cwt[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type, cwt_level=num_levels, cwt_scale=top_scale)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_val_acc += TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type)
print ('best validation is :{}'.format(best_val_acc))

## 3. Do K-Fold training and validation with windowed augmentation

Did not re-initialize RNN for every fold

###Whiten and center data

In [0]:
X_scaled = scale_data(X_train_valid)
train_fold, val_fold = Train_Val_Data(X_scaled, y_train_valid)
X_train_valid[train_fold[0]].shape

(1692, 22, 1000)

###Run the thing

In [0]:
aug_type = "window"
window_size = 200
vote_num = 20
best_val_acc = 0.0
model, criterion, optimizer = InitRNN(rnn_type='LSTM')
k=0
# indicate hyperparameters here
print ('fold {}'.format(k+1))
X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
best_val_acc += TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type) / 5
print ('average best validation accuracy of 5 folds is :{}'.format(best_val_acc))

fold 1
epoch: 1      time: 539.78    loss: 731.980    train acc: 0.270    val acc: 0.291
epoch: 2      time: 204.58    loss: 729.750    train acc: 0.283    val acc: 0.293
epoch: 3      time: 201.08    loss: 727.745    train acc: 0.293    val acc: 0.312
epoch: 4      time: 188.42    loss: 724.522    train acc: 0.302    val acc: 0.329
epoch: 5      time: 193.51    loss: 728.958    train acc: 0.287    val acc: 0.324
epoch: 6      time: 191.84    loss: 726.361    train acc: 0.295    val acc: 0.355
epoch: 7      time: 191.39    loss: 724.376    train acc: 0.303    val acc: 0.274
epoch: 8      time: 191.68    loss: 725.553    train acc: 0.297    val acc: 0.319
average best validation accuracy of 5 folds is :0.07092198581560284


In [0]:
X_test, y_test, p_test = X_test, y_test, person_test
if aug_type == 'window':
    EEG_testset = EEG_Dataset(X_train, y_train, p_train, X_val, y_val, p_val, X_test, y_test, p_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].permute(2, 0, 1).to(device)
        y = batch['y'].to(device)
        vote_idx = np.random.choice(1000-window_size, vote_num)
        vote_pred = np.zeros(y.shape[0])
        for i in range(len(vote_idx)):
            X_sub = X[vote_idx[i]:vote_idx[i]+200,:,:]
            output = model(X_sub)
            pred = torch.argmax(output, dim=1)
            if i == 0:
                vote_matrix = np.asarray(pred.cpu().view(-1, 1))
            else:
                vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
            for row in range(y.shape[0]):
                vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
        vote_pred = torch.from_numpy(vote_pred).long()
        correct += torch.sum(vote_pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total 
else:
    X_test, y_test, p_test = Aug_Data(X_test, y_test, p_test)
    EEG_testset = EEG_Dataset(X_test=X_test, y_test=y_test, p_test=p_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)    
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].permute(2, 0, 1).to(device)
        y = batch['y'].to(device)
        output = model(X)                    
        pred = torch.argmax(output, dim=1)
        correct += torch.sum(pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total
print ('Testing Accuracy: {:.4f}'.format(test_acc))


RuntimeError: ignored